<a href="https://colab.research.google.com/github/saisampaththumati-datascientists/Gen_AI/blob/main/summary_diloges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#@title Run this cell to setup Unsloth on Colab
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-deps  "xformers<0.0.27" "trl<0.9.0" peft==0.12.0 accelerate==0.32.1 bitsandbytes==0.43.2

#download datasets evaluate rouge_score and bert score
!pip install -q datasets==2.16.1 evaluate==0.4.1 rouge_score==0.1.2 bert_score==0.3.12
!pip install triton
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -U xformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached xformers-0.0.27.post2-cp310-cp310-manylinux2014_x86_64.whl.metadata (1.0 kB)
Using cached xformers-0.0.27.post2-cp310-cp310-manylinux2014_x86_64.whl (20.8 MB)
  Attempting uninstall: xformers
    Found existing installation: xformers 0.0.26.post1
    Uninstalling xformers-0.0.26.post1:
      Successfully uninstalled xformers-0.0.26.post1


In [2]:
import pandas as pd
import numpy as np
import huggingface_hub
from huggingface_hub import hf_hub_download
import torch
import evaluate

from datasets import load_dataset
from tqdm import tqdm
import unsloth
from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
dataset = load_dataset("knkarthick/dialogsum")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
test_data=30

In [6]:
test_dataset=dataset["test"].shuffle(seed=43).select(range(test_data))

In [9]:
test_diloges=[sample["dialogue"] for sample in test_dataset]
test_summary=[sample["summary"] for sample in test_dataset]

In [11]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    max_seq_length=4096,
    dtype=None,
    load_in_4bit=True)

==((====))==  Unsloth 2024.8: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [12]:
FastLanguageModel.for_inference(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((40

In [16]:
summarization_prompt_template = """[INST]
Summarize the summary mentioned in the user input. Be specific and concise in your dialogue.
Ensure that you retain the entities mentioned in the summary in your dialogue.

### User Input:
{summary}
[/INST]"""

In [17]:
prompt_prediction =[]

In [18]:
for golden_sample in tqdm(test_summary):
  try:
    prompt=summarization_prompt_template.format(summary=golden_sample)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output= model.generate(
            **inputs,
            max_new_tokens=128,
            use_cache=True,
            temperature=0,
            pad_token_id=tokenizer.eos_token_id

    )
    prediction = tokenizer.decode(
            output[0][inputs.input_ids.shape[-1]:],
            skip_special_tokens=True,
            cleanup_tokenization_spaces=True
        )
    prompt_prediction.append(prediction)

  except Exception as e:
        print(e) # log error and continue
        continue

100%|██████████| 30/30 [01:36<00:00,  3.23s/it]


In [32]:
prompt_prediction[1]

'Mr. Simpson is having lunch with Person 1 on Thursday.'

In [28]:
bert_scorer = evaluate.load("bertscore")

In [29]:
score = bert_scorer.compute(
    predictions=prompt_prediction,
    references=test_diloges,
    lang='en',
    rescale_with_baseline=True
)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
sum(score['f1'])/len(score['f1'])

0.053027458194022374

In [23]:
test_diloges[1]

"#Person1#: Mr. Simpson, if you are free, how about a lunch?\n#Person2#: When do you have in mind?\n#Person1#: How about Thursday?\n#Person2#: That will be fine with me.\n#Person1#: I know of a place you'll like and you have got to be there.\n#Person2#: Good. I'll be there at 12:30.\n#Person1#: OK, see you then.\n#Person2#: See you."

In [33]:
train_size, validation_size = 500, 100
training_dataset = dataset['train'].shuffle(seed=42).select(range(train_size))
validation_dataset = dataset['validation'].shuffle(seed=42).select(range(validation_size))

In [34]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [40]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    max_seq_length=4096,
    dtype=None,
    load_in_4bit=True)

==((====))==  Unsloth 2024.8: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [41]:
EOS_TOKEN = tokenizer.eos_token

In [43]:
def prompt_formatter(example, prompt_template):
    instruction='Write a concise diloges of the following summary.'
    dialogue=example["dialogue"]
    summary=example["summary"]

    formatted_prompt = prompt_template.format(instruction,summary, dialogue) + EOS_TOKEN

    return {'formatted_prompt': formatted_prompt}


In [44]:
formatted_training_dataset = training_dataset.map(
    prompt_formatter,
    fn_kwargs={'prompt_template': alpaca_prompt}
)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [45]:
formatted_validation_dataset = validation_dataset.map(
    prompt_formatter,
    fn_kwargs={'prompt_template': alpaca_prompt}
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [46]:
formatted_training_dataset[0]

{'id': 'train_6155',
 'dialogue': "#Person1#: Hello, Anna speaking!\n#Person2#: Hey, Anna, this is Jason.\n#Person1#: Jason, where have you been hiding lately? You know it's been a long time since your last call. Have you been good?\n#Person2#: Yes. How are you, Anna?\n#Person1#: I am fine. What have you been doing?\n#Person2#: Working. I've been really busy these days. I got a promotion.\n#Person1#: That's great, congratulations!\n#Person2#: Thanks. I am feeling pretty good about myself too. You know, bigger office, a raise and even an assistant.\n#Person1#: That's good. So I guess I'll have to make an appointment to see you.\n#Person2#: You are kidding.\n#Person1#: How long have you been working there?\n#Person2#: A bit over two years. This is a fast-moving company, and seniority isn't the only factor in deciding promotions.\n#Person1#: How do you like your new boss?\n#Person2#: She is very nice and open-minded.\n#Person1#: Much better than the last one, huh?\n#Person2#: Yeah. He was

In [47]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    use_gradient_checkpointing=True,
    random_state=42,
    loftq_config=None
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [48]:
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_training_dataset,
    eval_dataset=formatted_validation_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    dataset_text_field = "formatted_prompt",
    max_seq_length=2048,
    dataset_num_proc=2,
    packing=False, # Increases efficiency for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=20,
        evaluation_strategy="epoch",
        save_strategy='epoch',
        metric_for_best_model="eval_loss",
        load_best_model_at_end=True,
        greater_is_better=False,
        learning_rate=5e-5,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="paged_adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir="outputs"
    )
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

In [49]:

training_history = trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 500 | Num Epochs = 20
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,240
 "-____-"     Number of trainable parameters = 41,943,040
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss
0,1.237300,1.149591
2,0.987900,1.153851
4,0.594000,1.368912


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-pac

In [50]:
training_history

TrainOutput(global_step=312, training_loss=0.958841206649175, metrics={'train_runtime': 3196.2255, 'train_samples_per_second': 3.129, 'train_steps_per_second': 0.388, 'total_flos': 4.1724710393856e+16, 'train_loss': 0.958841206649175, 'epoch': 4.992})

In [51]:
test_dataset = dataset['test']

In [52]:
instruction='Write a concise dialogue of the following summary.'
test_dialogue = test_dataset[0]['dialogue']
test_summary = test_dataset[0]['summary']

In [53]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(

In [54]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction,
        test_summary,
        "", # leave output blank for generation
    )
], return_tensors="pt").to("cuda")

In [55]:
outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    use_cache=True,
    pad_token_id=tokenizer.eos_token_id
)

In [56]:
print(
    tokenizer.decode(
        outputs[0][inputs.input_ids.shape[-1]:],
        skip_special_tokens=True,
        cleanup_tokenization_spaces=True
    )
)

#Person1#: Ms. Dawson, I need to write a memo to inform every employee that we have to change the communication method.
#Person2#: What do you mean?
#Person1#: We have to stop using Instant Messaging anymore.
#Person2#: Why?
#Person1#: Because it is not secure.
#Person2#: What should we use instead?
#Person1#: We should use email.
#Person2#: OK. I will write the memo right now.
#Person1#: Good.


In [58]:
test_dialogue

"#Person1#: Ms. Dawson, I need you to take a dictation for me.\n#Person2#: Yes, sir...\n#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?\n#Person2#: Yes, sir. Go ahead.\n#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.\n#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?\n#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.\n#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.\n#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please continue with the

In [59]:
import locale

def getpreferredencoding():
    return "UTF-8"

locale.getpreferredencoding = getpreferredencoding

In [60]:
lora_model_name = "dialogue-summarizer-mistral"

In [61]:
model.save_pretrained(lora_model_name)

In [62]:
ls -lh {lora_model_name}

total 161M
-rw-r--r-- 1 root root  741 Sep  5 19:42 adapter_config.json
-rw-r--r-- 1 root root 161M Sep  5 19:42 adapter_model.safetensors
-rw-r--r-- 1 root root 5.0K Sep  5 19:42 README.md


In [63]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [64]:
!cp -r {lora_model_name} /content/drive/MyDrive



In [65]:
import evaluate

from google.colab import drive
from datasets import load_dataset
from unsloth import FastLanguageModel
from tqdm import tqdm

In [66]:
lora_model_name = "dialogue-summarizer-mistral"

In [67]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
!cp -r /content/drive/MyDrive/{lora_model_name} .

In [74]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=lora_model_name,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
    device_map="auto",  # 'auto' will attempt to fit the model on the GPU and offload to CPU/disk if needed
)

==((====))==  Unsloth 2024.8: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1381: UserWarning: Current model requires 75497472.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 